In [150]:
# Import libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Read data into pandas dataframe
kick_data = pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter002.csv")
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter003.csv"))
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter004.csv"))
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter005.csv"))
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter006.csv"))
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter007.csv"))
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter008.csv"))
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter009.csv"))
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter010.csv"))
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter011.csv"))
kick_data = kick_data.append(pd.read_csv("/Users/Nicole/Documents/Kickstarter/Kickstarter012.csv"))

# Isolate relevant attributes
kick_data = kick_data.loc[:, ['backers_count','blurb','category','country', 'goal', 
                              'staff_pick', 'state', 'usd_pledged']]

# remove live entries as their final state is unknown
kick_data = kick_data[kick_data.state != 'live']

# remove rows with missing values
kick_data = kick_data.dropna()

# reset index for model-building and to remove duplicate values
kick_data = kick_data.reset_index(drop=True)

# initial count will return 40169 for most categories, 40167 for blurb column
kick_data.count()

kick_data.tail()

,backers_count,blurb,category,country,goal,staff_pick,state,usd_pledged
40162,149,A cheeky little card game of face-stuffing ham...,"{""id"":273,""name"":""Playing Cards"",""slug"":""games...",GB,1500.0,False,successful,4153.527953
40163,44,"A high-quality, hand-built tap tempo delay ped...","{""id"":339,""name"":""Sound"",""slug"":""technology/so...",CA,8000.0,True,successful,11266.636702
40164,473,A Superhero RPG for the 21st Century!,"{""id"":34,""name"":""Tabletop Games"",""slug"":""games...",US,28000.0,False,successful,34332.500000
40165,52,"The new series ""Fabulous High"" needs your back...","{""id"":33,""name"":""Webseries"",""slug"":""film & vid...",US,5000.0,False,successful,6510.000000
40166,41,"Low-brow art, stickers and prints influenced b...","{""id"":22,""name"":""Illustration"",""slug"":""art/ill...",US,1200.0,False,successful,1367.000000


In [149]:
# kNN model

z = 32000

# feature variables
x_train = kick_data.loc[:z, ['backers_count','goal', 'staff_pick', 'usd_pledged']]
x_test = kick_data.loc[z:, ['backers_count','goal', 'staff_pick', 'usd_pledged']]
# response variable
y_train = kick_data.loc[:z, 'state'] 
y_test = kick_data.loc[z:, 'state']

from sklearn.neighbors import KNeighborsClassifier 
from sklearn import metrics
import math as m

# find best k
best_k = 0
best_accuracy = 0

for i in range (1, m.floor(len(x_test)/50)):
    # training model on training set
    knn = KNeighborsClassifier(n_neighbors=i) 
    knn.fit(x_train, y_train) 
    # making predictions on testing set 
    y_pred = knn.predict(x_test) 
    a = metrics.accuracy_score(y_test, y_pred)
    if a > best_accuracy:
        best_k = i
        best_accuracy = a
print(best_k, "," ,best_accuracy)

9 , 0.929472266438
